# T5.1 - Long Range Interactions

Let's create a system to examine both how ewald summation works, as well as a sketch of how FMMs might work. Again, this is just for illustrative purposes, as the full treatment of either depends heavily on the periodicity of the problem, as well as the type of long-range interaction you are evalulating.

Let's consider a system of $N$ particles in a 3D system.

In [1]:
using Revise;
using Plots;
using LaTeXStrings;
using StaticArrays;
using Random;
using Distributions;
Random.seed!(1234);

In [2]:
# We are going to build out own entire system for MD in the notebook so that we can do whatever we want with it.
# Make it fairly general.

# MD System keeps track of the current system state, and things that don't change per particle
struct MDSystem
    N::Int64;
    X::Vector{SVector{3,Float64}};
    P::Vector{SVector{3,Float64}};
    mass::Vector{Float64};
    charge::Vector{Float64};
    Ekin::Float64;
    Epot::Float64;
    Etot::Float64;
    kT::Float64;
end
# Create a function to create a default version of this structure
function MDSystem(N::Int)
    X = Vector{SVector{3,Float64}}(undef, N);
    P = Vector{SVector{3,Float64}}(undef, N);
    # Initialize X and P to zeros (thermalize and set initial positions later)
    for idx in 1:N
        X[idx] = SVector{3,Float64}(0.0, 0.0, 0.0);
        P[idx] = SVector{3,Float64}(0.0, 0.0, 0.0);
    end
    mass = zeros(N);
    charge = zeros(N);
    Ekin = 0.0;
    Epot = 0.0;
    Etot = 0.0;
    kT = 0.0;
    return MDSystem(N, X, P, mass, charge, Ekin, Epot, Etot, kT);
end

# UnitCell keeps track of the dimensionality of the system.
struct UnitCell
    L::SVector{3,Float64};
    periodic::SVector{3,Int64};
end
function UnitCell(L::Float64, periodicX::Bool, periodicY::Bool, periodicZ::Bool)
    Lvec = SVector{3,Float64}(L, L, L);
    periodicvec = SVector{3,Int64}(periodicX, periodicY, periodicZ);
    return UnitCell(Lvec, periodicvec);
end

UnitCell

In [7]:
# This will randomize our initial positions inside of a cell of length L
function RandomInitialPositionsUnitCell!(mdsys::MDSystem,
    unitcell::UnitCell)
    for idx in 1:mdsys.N
        randpos = (rand(3) .- 0.5) * unitcell.L;
        mdsys.X[idx] = SVector{3,Float64}(randpos[1], randpos[2], randpos[3]);
    end
    return nothing
end
# This will thermalize the 1D system (assign velocities in a Maxwellian distribution way)
function ThermalizeMomenta!(mdsys::MDSystem,
        m::Float64,
        kT::Float64)
    N = mdsys.N;
    MaxwellDistribution = Normal(0.0, sqrt(kT/m));
    for idx in 1:N
        mdsys.P[idx] = rand(MaxwellDistribution, 3);
    end
    return nothing
end

ThermalizeMomenta! (generic function with 1 method)

In [8]:
# Initialize our system
N = 10;
L = 100.0;
mass = 1.0;
kT = 1.0;
mdsys = MDSystem(N);
unitcell = UnitCell(L, true, true, true);
RandomInitialPositionsUnitCell!(mdsys, unitcell);
# ThermalizeMomenta!(mdsys, mass, kT);

LoadError: MethodError: no method matching *(::Vector{Float64}, ::SVector{3, Float64})
The function `*` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m)
[0m[90m   @[39m [90mBase[39m [90m[4moperators.jl:596[24m[39m
[0m  *([91m::StaticArray{Tuple{N, M}, T, 2} where {N, M, T}[39m, ::StaticArray{Tuple{N}, T, 1} where {N, T})
[0m[90m   @[39m [36mStaticArrays[39m [90m~/.julia/packages/StaticArrays/9Yt0H/src/[39m[90m[4mmatrix_multiply.jl:10[24m[39m
[0m  *([91m::PDMats.PDiagMat[39m, ::AbstractVector)
[0m[90m   @[39m [35mPDMats[39m [90m~/.julia/packages/PDMats/cAM9h/src/[39m[90m[4mpdiagmat.jl:55[24m[39m
[0m  ...
